## Scenario 3: Multiple data scientists working on multiple ML models

MLflow setup:
* Tracking server: yes, remote server (EC2).
* Backend store: postgresql database.
* Artifacts store: s3 bucket.

The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file `mlflow_on_aws.md` to create a new AWS account and launch the tracking server. 

In [1]:
import mlflow
import os

# os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "ec2-72-44-45-204.compute-1.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_artifact_uri()}")

tracking URI: 's3://mlflow-artifacts-s3-bucket/0/4c7f9e0e7d774d79b70588bd70a15def/artifacts


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-s3-bucket/3', creation_time=1716246279952, experiment_id='3', last_update_time=1716246279952, lifecycle_stage='active', name='my-experiment-3', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-s3-bucket/0', creation_time=1716224770686, experiment_id='0', last_update_time=1716224770686, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
mlflow.end_run()

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-3")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

/home/codespace/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


default artifacts URI: 's3://mlflow-artifacts-s3-bucket/3/315def39b5c8464999debeaa87ce8205/artifacts'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-artifacts-s3-bucket/3', creation_time=1716246279952, experiment_id='3', last_update_time=1716246279952, lifecycle_stage='active', name='my-experiment-3', tags={}>,
 <Experiment: artifact_location='s3://mlflow-artifacts-s3-bucket/0', creation_time=1716224770686, experiment_id='0', last_update_time=1716224770686, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [8]:
client.search_registered_models()

[]

In [11]:
run_id = client.search_runs(experiment_ids='0')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name="iris-classifier"
)

# run_id = client.search_runs(experiment_ids='1')[0].info.run_id

# mlflow.register_model(
#     model_uri=f"runs:/{run_id}/models",
#     name='iris-classifier'
# )

Successfully registered model 'iris-classifier'.
2024/05/20 23:25:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716247558160, current_stage='None', description='', last_updated_timestamp=1716247558160, name='iris-classifier', run_id='4c7f9e0e7d774d79b70588bd70a15def', run_link='', source='s3://mlflow-artifacts-s3-bucket/0/4c7f9e0e7d774d79b70588bd70a15def/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>